In [5]:
tweets_file = "data/train_tweets.txt"
authorList = []
with open(tweets_file, "r") as file:
    for i in file:
        temp = i.split("\t")
        authorList.append(temp[0])
        

In [6]:
print(len(authorList))

328932


In [7]:
authorList = list(set(authorList))

In [8]:
print(len(authorList))

9297


In [10]:
text = "handle handle I just think that developmentally speaking, a lot of kids (not all, of course), might not be ready for that."

In [11]:
import nltk
tokenized_sentence = nltk.tokenize.word_tokenize(text)
preprocess_sentence = [for word in tokenized_sentence if type(word)]
print(tokenized_sentence)

['handle', 'handle', 'I', 'just', 'think', 'that', 'developmentally', 'speaking', ',', 'a', 'lot', 'of', 'kids', '(', 'not', 'all', ',', 'of', 'course', ')', ',', 'might', 'not', 'be', 'ready', 'for', 'that', '.']


In [18]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0902 23:52:29.824620 4740359616 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


###

In [19]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [20]:
tweets_file = "data/train_tweets.txt"
authorList = []
with open(tweets_file, "r") as file:
    for i in file:
        temp = i.split("\t")
        authorList.append(temp[0])

In [21]:
label_list = [int(i) for i in list(set(authorList))]
print(label_list)

[7953, 1410, 7414, 6890, 4337, 5381, 8019, 9800, 5419, 8339, 5662, 684, 5799, 2543, 6190, 6406, 5131, 3085, 6084, 9581, 7723, 9330, 8416, 2788, 7316, 6354, 1205, 4586, 6959, 3071, 7096, 1456, 1785, 6970, 6, 8130, 5491, 2309, 2653, 582, 8480, 5076, 8126, 183, 3729, 2437, 100, 2392, 2149, 3297, 4188, 5735, 1697, 7765, 9803, 2918, 7235, 789, 8372, 9392, 3772, 2959, 7380, 812, 6127, 8263, 1153, 1299, 9326, 4089, 7357, 2549, 6637, 7125, 3168, 5712, 1420, 1853, 5547, 5890, 8894, 2440, 256, 9006, 5738, 2723, 7673, 1030, 9100, 2255, 5957, 2631, 6160, 8922, 3158, 5766, 2410, 8534, 9220, 2394, 8407, 4, 4747, 9970, 5145, 5861, 3123, 412, 6109, 9190, 43, 2585, 4074, 6572, 3286, 9869, 6257, 1372, 1172, 6981, 5290, 3790, 7062, 5789, 8822, 741, 5603, 2906, 7373, 6726, 9526, 1352, 1015, 1632, 524, 4416, 6728, 8502, 5121, 8791, 7288, 9293, 4847, 609, 7579, 5468, 888, 8124, 6353, 86, 9210, 8954, 4981, 265, 3094, 5754, 8265, 4342, 2445, 8039, 422, 3521, 5757, 399, 2469, 2790, 8169, 2397, 3917, 163, 5475,

In [23]:
train = pd.read_csv("data/train_data.csv")
test = pd.read_csv("data/dev_data.csv")

In [24]:
#print(tweet_df["tweet"])
train["tweet"] = train["tweet"].astype(str)
test["tweet"] = test["tweet"].astype(str)
print(train["tweet"])

0         handle I miss you Hun. I've been grindin. Imma...
1         handle Oh it happened...I saw the belt on the ...
2         won a signed poster and tkts to handle at hand...
3                                    I want Lisa's earrings
4                                   handle Wellllll... http
5         Absolutely loving this! Make sure to check out...
6         Can babies play on the computer? This answers ...
7         handle Birdman, Jay Sean & Mack Maine performe...
8         Jim Wenzloff linked here as a comment on the b...
9         Got a cool Startup Weekend Corvallis card from...
10        Plenty of buses in ATL... take a step off the ...
11                                          handle thanks!!
12        Yes, Important! RT handle Seeing student appli...
13        40% off 3.5-Star Washington DC Hotels This Wee...
14                                       handle poetry? LOL
15        Are You an Ex Boyfriend Who is Hanging On Tips...
16        Poped 50mg should be out with 

In [25]:
DATA_COLUMN = 'tweet'
LABEL_COLUMN = 'author_Id'

In [26]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [27]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0902 23:56:20.427638 4740359616 deprecation.py:323] From /anaconda/envs/deepTF/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0902 23:56:22.614618 4740359616 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [28]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 246699


I0902 23:56:23.514510 4740359616 run_classifier.py:774] Writing example 0 of 246699


INFO:tensorflow:*** Example ***


I0902 23:56:23.523557 4740359616 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0903 00:03:17.529464 4740359616 run_classifier.py:774] Writing example 60000 of 82233


INFO:tensorflow:Writing example 70000 of 82233


I0903 00:03:30.165437 4740359616 run_classifier.py:774] Writing example 70000 of 82233


INFO:tensorflow:Writing example 80000 of 82233


I0903 00:03:42.720469 4740359616 run_classifier.py:774] Writing example 80000 of 82233


In [29]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


In [30]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [31]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [32]:
# Compute # train and warmup steps from batch size
#num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_train_steps = 3000
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
print(num_train_steps)

3000


In [33]:
OUTPUT_DIR = "model"

In [34]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [35]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13bbcdb70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0903 00:03:45.949602 4740359616 estimator.py:201] Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13bbcdb70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [37]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0903 00:05:46.165603 4740359616 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0903 00:05:50.779899 4740359616 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:global_step/sec: 0.0220131


I0903 12:28:52.031723 4740359616 basic_session_run_hooks.py:680] global_step/sec: 0.0220131


INFO:tensorflow:loss = 8.890215, step = 1001 (4542.594 sec)


I0903 12:28:52.089339 4740359616 basic_session_run_hooks.py:247] loss = 8.890215, step = 1001 (4542.594 sec)


KeyboardInterrupt: 

In [38]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [39]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0903 12:31:26.214693 4740359616 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0903 12:31:29.011863 4740359616 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


I0903 12:31:51.225540 4740359616 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0903 12:31:51.768018 4740359616 session_manager.py:493] Done running local_init_op.


InvalidArgumentError: assertion failed: [predictions must be in [0, 1]] [Condition x <= y did not hold element-wise:x (f1/remove_squeezable_dimensions/cond/Merge:0) = ] [4412 4412 4765...] [y (f1/Cast_1:0) = ] [1]
	 [[node f1/assert_less_equal/Assert/Assert (defined at <ipython-input-30-2e070f5d79cc>:30) ]]

Caused by op 'f1/assert_less_equal/Assert/Assert', defined at:
  File "/anaconda/envs/deepTF/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/envs/deepTF/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda/envs/deepTF/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda/envs/deepTF/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda/envs/deepTF/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/an

In [40]:
print(estimator)

In [101]:
print(len(label_list))


9297


In [93]:
def getPrediction(in_sentences):
  labels = label_list
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 2) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [ prediction['labels'] for sentence, prediction in zip(in_sentences, predictions)]

In [94]:
pred_sentences = [
  "The quasi dollar cab type shuttle dude is mad cuz I'm txting and not talking to him! LOL",
  "Melissa's cuban accent makes me laugh. It's the cutest thing and she doesn't even realize she has an accent. Haha"
    
]

In [95]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 2


I0903 23:50:50.176330 4740359616 run_classifier.py:774] Writing example 0 of 2


INFO:tensorflow:*** Example ***


I0903 23:50:50.181963 4740359616 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0903 23:50:55.573920 4740359616 saver.py:1270] Restoring parameters from model/model.ckpt-1000


INFO:tensorflow:Running local_init_op.


I0903 23:50:56.511170 4740359616 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0903 23:50:56.664143 4740359616 session_manager.py:493] Done running local_init_op.


In [96]:
print(predictions)

[4765, 4547]


In [97]:
pred_sentences = list(test["tweet"])


In [98]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 82233


I0903 23:51:28.532528 4740359616 run_classifier.py:774] Writing example 0 of 82233


INFO:tensorflow:*** Example ***


I0903 23:51:28.539824 4740359616 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0903 23:53:55.873439 4740359616 saver.py:1270] Restoring parameters from model/model.ckpt-1000


INFO:tensorflow:Running local_init_op.


I0903 23:54:00.817904 4740359616 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0903 23:54:01.111325 4740359616 session_manager.py:493] Done running local_init_op.


In [100]:
author_id = list(test["author_Id"])
count = 0
for i in range(len(predictions)):
    if predictions[i] == author_id[i]:
        count += 1
print(count/len(predictions))

0.0001337662495591794


In [103]:
print(predictions)

[2818, 4412, 4765, 3926, 4043, 44, 3071, 4043, 5296, 44, 44, 4043, 4043, 4412, 596, 4043, 4412, 44, 4043, 2818, 4547, 4412, 3926, 4043, 4043, 4043, 4043, 2143, 3926, 4132, 780, 2293, 4765, 4043, 6901, 4175, 2818, 8440, 4604, 4412, 2977, 4043, 4765, 3071, 2977, 4604, 4043, 2143, 2977, 4043, 18, 4043, 4043, 4175, 4043, 4765, 18, 4043, 4765, 5761, 4043, 4765, 4043, 3926, 4175, 2391, 4043, 4043, 44, 4043, 4043, 4412, 4043, 4175, 5761, 2818, 4043, 3926, 4175, 4043, 1716, 2964, 4043, 5296, 2977, 2964, 4043, 4043, 4043, 4043, 4043, 4043, 780, 2977, 4043, 2977, 596, 4765, 2143, 4765, 5263, 4412, 2143, 4043, 4043, 3273, 4043, 4412, 4043, 2818, 4043, 44, 1442, 4765, 2143, 44, 2964, 4043, 2143, 5296, 4043, 2818, 2964, 3071, 3071, 4043, 4175, 4765, 4043, 4043, 4765, 780, 2818, 4175, 4043, 5434, 2818, 2143, 3926, 2818, 3031, 4043, 4412, 2818, 3926, 780, 4412, 4043, 4043, 4175, 4412, 4043, 8875, 7670, 5296, 2977, 2818, 4043, 780, 2818, 4043, 9036, 4043, 4043, 5761, 5296, 4043, 44, 44, 4043, 2143, 69

In [104]:
print(author_id)

[7597, 7918, 2125, 6073, 757, 2511, 417, 8691, 5891, 6140, 418, 5568, 384, 678, 1483, 4551, 6409, 8277, 7883, 9341, 4746, 3809, 9258, 4318, 4094, 2755, 8305, 3566, 1759, 145, 8511, 3638, 9396, 8331, 8663, 1246, 4037, 1993, 673, 5432, 4213, 6911, 4166, 8564, 8965, 6188, 2358, 6393, 2749, 3023, 4952, 1239, 5951, 6720, 6058, 564, 2262, 3566, 5095, 2550, 154, 4285, 9629, 5873, 5066, 7139, 7892, 6791, 544, 7202, 2702, 9454, 4130, 5651, 7808, 6751, 3472, 7866, 8797, 4758, 5661, 9455, 6826, 6579, 2484, 36, 6760, 368, 4806, 6761, 5185, 6428, 2533, 5712, 7610, 1175, 1731, 8514, 2785, 9220, 6807, 1340, 4868, 6760, 7473, 9830, 2970, 9328, 3377, 941, 1192, 7778, 1650, 4925, 6703, 3816, 6064, 2484, 7140, 8064, 5878, 3289, 8428, 2229, 256, 3017, 9728, 6323, 8244, 1768, 2435, 8601, 2785, 4919, 7398, 9554, 3506, 9350, 2081, 1553, 2065, 8930, 2252, 9712, 7810, 3688, 8438, 8049, 9853, 6517, 1463, 5043, 6442, 5913, 2793, 8872, 3046, 1175, 703, 541, 8454, 9474, 7737, 1342, 2279, 1906, 4727, 5845, 990, 298

In [107]:
print(test.author_id == 4043)

AttributeError: 'DataFrame' object has no attribute 'author_id'

In [113]:
from collections import Counter

c =Counter(predictions)

In [114]:
print(c)

Counter({4043: 26011, 2818: 7641, 4412: 6292, 2143: 6009, 2977: 4788, 4765: 4296, 44: 3521, 4175: 3140, 780: 2712, 5296: 2453, 3926: 2386, 2964: 945, 18: 843, 4604: 814, 5263: 751, 3071: 690, 5106: 544, 6901: 531, 915: 432, 2207: 411, 2293: 399, 6392: 378, 5448: 306, 7837: 302, 4547: 298, 4990: 275, 5027: 267, 596: 225, 3668: 221, 4679: 217, 2873: 209, 3273: 190, 6178: 183, 5214: 146, 7670: 142, 5761: 141, 4906: 136, 2617: 129, 3093: 125, 5434: 123, 8875: 121, 7536: 102, 6785: 101, 3399: 99, 6540: 87, 7055: 73, 5579: 69, 7214: 68, 3705: 67, 4262: 60, 8208: 59, 7313: 51, 5976: 51, 1172: 49, 8218: 48, 1401: 44, 7767: 43, 1716: 40, 2849: 40, 5410: 40, 6675: 37, 4458: 36, 784: 34, 6410: 33, 5595: 32, 5517: 30, 2965: 27, 2391: 26, 3352: 26, 4141: 25, 5807: 25, 1215: 23, 209: 21, 1960: 21, 652: 20, 2269: 20, 8440: 19, 6191: 19, 3269: 19, 8915: 19, 4835: 18, 1629: 17, 9036: 16, 4381: 15, 116: 14, 5720: 14, 1225: 14, 3979: 14, 3432: 13, 2508: 13, 8275: 13, 3008: 13, 705: 12, 3334: 12, 2817: 12

In [115]:
print(len(c))

277


In [116]:
print(len(set(author_id)))

9184
